In [ ]:
%pip install -q tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import yfinance as yf
from sklearn import preprocessing


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 16, 10

### Get data

In [ ]:
data=yf.download("GOOGL", period="1y", interval="1d")
df = pd.DataFrame(data)
df.head()

#Remove unnecessary columns
df.drop(columns=[
    'Adj Close'],
    inplace=True)

df.head()

### Convert data/time to useable format and normalize data

In [ ]:
date_time = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S')

timestamp_s = date_time.map(pd.Timestamp.timestamp)

day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

df.head()

### data preprocessing

In [ ]:
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_norm = pd.DataFrame(x_scaled, columns = df.columns)

df_norm.head()

### split data into train and test

In [ ]:
train_size = int(len(df_norm) * 0.8)
test_size = len(df_norm) - train_size
train, test = df_norm.iloc[0:train_size], df_norm.iloc[train_size:len(df_norm)]
print(len(train), len(test))

### create dataset function

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

### creeate train/test datasets 

In [ ]:
time_steps = 10

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train.Close, time_steps)
X_test, y_test = create_dataset(test, test.Close, time_steps)

print(X_train.shape, y_train.shape)

In [ ]:
X_train[0]

In [ ]:
y_train[0]

### create model

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Conv1D(filters=5, kernel_size=3, activation='relu', padding='same', input_shape=(X_train.shape[0], X_train.shape[1], X_train.shape[2])))
model.add(keras.layers.MaxPooling1D(pool_size=2))
model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))
model.add(keras.layers.LSTM(128, stateful=True, return_sequences=True))
model.add(keras.layers.LSTM(32, stateful=True))
model.add(keras.layers.Dense(1))
model.compile(loss='mae', optimizer=keras.optimizers.Adam(0.001), metrics=['mae', 'mape', 'acc'])

### training

In [ ]:
EPOCHS=500

# Create early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    start_from_epoch=50,
    verbose=1
)

# create reduce lr on plateau callback
reduce_lr_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.9,
    monitor='val_loss',
    patience=15,
    min_lr=0.001,
    verbose=1
)

history = model.fit(
    X_train, y_train, 
    epochs=EPOCHS, 
    validation_split=0.1, 
    callbacks=[early_stopping, reduce_lr_plateau],
    verbose=1, 
    shuffle=False
)

### evaluate

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train, 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test, marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();